Conclusion:

We need better data. Our average cross validation score is about .3, which is not very good at all. Thus we can say that the data is not a good predictor of scores. (The dataset is not incredibly long, taking only 1.76 seconds for the random forest to be generated.)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("movies.csv", encoding='latin1')

In [3]:
df.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [4]:
df.describe()

,budget,gross,runtime,score,votes,year
count,6.820000e+03,6.820000e+03,6820.00000,6820.000000,6.820000e+03,6820.000000
mean,2.458113e+07,3.349783e+07,106.55132,6.374897,7.121952e+04,2001.000293
std,3.702254e+07,5.819760e+07,18.02818,1.003142,1.305176e+05,8.944501
min,0.000000e+00,7.000000e+01,50.00000,1.500000,2.700000e+01,1986.000000
25%,0.000000e+00,1.515839e+06,95.00000,5.800000,7.665250e+03,1993.000000
50%,1.100000e+07,1.213568e+07,102.00000,6.400000,2.589250e+04,2001.000000
75%,3.200000e+07,4.006534e+07,115.00000,7.100000,7.581225e+04,2009.000000
max,3.000000e+08,9.366622e+08,366.00000,9.300000,1.861666e+06,2016.000000


In [5]:
import statsmodels.formula.api as smf
linear_formula = 'score ~ budget+gross+runtime+votes+year'

# Fit the model to our data using the formula.
lm = smf.ols(formula=linear_formula, data=df).fit()

lm.pvalues

Intercept     1.107006e-04
budget        3.146332e-71
gross         9.134408e-01
runtime      1.225326e-191
votes        1.529742e-174
year          1.486909e-08
dtype: float64

In [6]:
lm.rsquared

0.28822445887934156

Budget does not have a P-value of .05 or less, so we'll exclude it.

In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

X = df[['budget', 'runtime', 'votes', 'year']]
y = df['score']

treeclf = DecisionTreeRegressor(max_depth=3, random_state=1)
treeclf.fit(X, y)

# create a Graphviz file
export_graphviz(treeclf, out_file='df.dot')

Graphviz is having issues with Jupyter Notebook. Thus the file is downloaded and input into an external source.


Copy & Paste the following into http://www.webgraphviz.com/ to see results:

digraph Tree {
node [shape=box] ;
0 [label="X[2] <= 110.5\nmse = 1.006\nsamples = 6820\nvalue = 6.375"] ;
1 [label="X[3] <= 94647.5\nmse = 0.991\nsamples = 4661\nvalue = 6.137"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="X[0] <= 11850000.0\nmse = 0.968\nsamples = 4029\nvalue = 6.026"] ;
1 -> 2 ;
3 [label="mse = 0.962\nsamples = 2559\nvalue = 6.192"] ;
2 -> 3 ;
4 [label="mse = 0.845\nsamples = 1470\nvalue = 5.736"] ;
2 -> 4 ;
5 [label="X[3] <= 259843.0\nmse = 0.564\nsamples = 632\nvalue = 6.843"] ;
1 -> 5 ;
6 [label="mse = 0.483\nsamples = 496\nvalue = 6.673"] ;
5 -> 6 ;
7 [label="mse = 0.365\nsamples = 136\nvalue = 7.464"] ;
5 -> 7 ;
8 [label="X[3] <= 250468.5\nmse = 0.652\nsamples = 2159\nvalue = 6.889"] ;
0 -> 8 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;
9 [label="X[0] <= 36500000.0\nmse = 0.585\nsamples = 1853\nvalue = 6.768"] ;
8 -> 9 ;
10 [label="mse = 0.52\nsamples = 1308\nvalue = 6.911"] ;
9 -> 10 ;
11 [label="mse = 0.571\nsamples = 545\nvalue = 6.423"] ;
9 -> 11 ;
12 [label="X[3] <= 607149.5\nmse = 0.425\nsamples = 306\nvalue = 7.625"] ;
8 -> 12 ;
13 [label="mse = 0.348\nsamples = 250\nvalue = 7.464"] ;
12 -> 13 ;
14 [label="mse = 0.14\nsamples = 56\nvalue = 8.343"] ;
12 -> 14 ;
}


In [8]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
import time

start_time = time.clock() # Start time.
rfr = ensemble.RandomForestRegressor()
print('Scores: ', cross_val_score(rfr, X, y, cv=10)) # Return cross validation scores.
print('Runtime: '+'%s seconds'% (time.clock() - start_time)) # End time.

Scores:  [0.16767058 0.17429468 0.2819332  0.31419471 0.38247358 0.42068989
 0.41616088 0.31235164 0.27837659 0.2239375 ]
Runtime: 1.7644123741478996 seconds
